In [ ]:
# Install required packages
!pip install -U langchain-community
!pip install -U langchain langchain-openai
!pip install faiss-cpu
!pip install openai==1.68.2 openai-agents
!pip install pydub
!pip install librosa 
!pip install soundfile
!pip install noisereduce
!pip install ollama
!pip install git+https://github.com/openai/whisper.git
!apt-get update && apt-get install -y ffmpeg

In [ ]:
# Import necessary libraries
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from pydub import AudioSegment, effects
from agents import Agent, Runner

import noisereduce as nr
import soundfile as sf
import librosa
import whisper
import asyncio
import ollama
import openai
import json
import math
import os

In [ ]:
# Audio processing and preprocessing
audio = AudioSegment.from_file("มิจฉาชีพ.wav")
audio = audio.set_channels(1).set_frame_rate(16000)
audio = effects.normalize(audio)
audio.export("normalized.wav", format="wav")
y, sr = librosa.load("normalized.wav", sr=16000)
reduced = nr.reduce_noise(y=y, sr=sr)
sf.write("preprocessed_มิจฉาชีพ.wav", reduced, sr)

# Audio Slitting into smaller chunks
audio = AudioSegment.from_wav("preprocessed_มิจฉาชีพ.wav")
chunk_length_ms = 60 * 1000  # 60 seconds
chunks = math.ceil(len(audio) / chunk_length_ms)

# model = whisper.load_model("base")
# model = whisper.load_model("small")
# model = whisper.load_model("medium")
model = whisper.load_model("large")
transcript = ""
overlap_ms = 2 * 1000

for i in range(chunks):
    if i == 0:
      chunk = audio[i*chunk_length_ms:(i+1)*chunk_length_ms]
    else:
      chunk = audio[i*chunk_length_ms - overlap_ms:(i+1)*chunk_length_ms]
    chunk_path = f"chunk_{i}.wav"
    chunk.export(chunk_path, format="wav")
    
    result = model.transcribe(chunk_path, language="th")
    transcript += f"{result['text']}"

# Display the output
print(transcript)

In [ ]:
# Set your key here or via environment variable
client = openai.OpenAI(api_key="YOUR_API_KEY")

prompt = f"""
The following is a messy, unstructured transcription of a phone call in Thai. It contains overlapping phrases, unclear speaker turns, and no punctuation.

Your task is to:
1. Structure the content as a readable conversation.
2. Add punctuation and line breaks to make it natural.
3. Keep the language in Thai and maintain the emotional tone.

Expected output format:
JSON format with key equals to 'transcription' and value as string type

Here is the transcription:
{transcript}
"""

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that restructures messy Thai call transcriptions."},
        {"role": "user", "content": prompt}
    ]
)

# Display the output
print(response.choices[0].message.content)
transcript = str(response.choices[0].message.content

In [ ]:
# Set your key here or via environment variable
client = openai.OpenAI(api_key="YOUR_API_KEY")

prompt = f"""
You are a QA agent. Evaluate the following call transcript.
Transcript:
{transcript}

Check for:
1. Proper greeting in Thai
2. Script adherence
3. Politeness by saying "ครับ"/"ค่ะ" in the conversation
4. Resolution provided

Give a score out of 10 for each criteria.

Expected output format:
JSON format with
1. key equals to 'greeting' and value as string type
2. key equals to 'script_adherence' and value as string type
3. key equals to 'politeness' and value as string type
4. key equals to 'resolution' and value as string type
"""

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant for contact center QA."},
        {"role": "user", "content": prompt}
    ]
)

# Display the output
print(response.choices[0].message.content)

In [ ]:
# Set your key here or via environment variable
embedding = OpenAIEmbeddings(openai_api_key="YOUR_API_KEY")

documents = [
    "Our company handles with Thai customer",
    "All agents/staffs must greet every customer in Thai language with 'สวัสดี'",
    "Customers should be treated with empathy.",
    "At the end of every dialogue response by agents/staffs must be ended with 'ครับ' or 'ค่ะ'",
]

# Create FAISS index from documents
faiss_index = FAISS.from_texts(documents, embedding)

llm = ChatOpenAI(
    openai_api_key="YOUR_API_KEY",
    model_name="gpt-4",
    temperature=0
)

qa_chain = RetrievalQA.from_chain_type(
   llm=llm,
   retriever=faiss_index.as_retriever()
)

# Now augment the prompt using retrieved knowledge
# Prepare the query for evaluation
query = f"Evaluate this transcript (in Thai Language): {transcript}"
# Display the output
print(qa_chain.invoke(query))

In [ ]:
# Set environment variables (e.g., API keys)
os.environ["YOUR_API_KEY"] = "YOUR_API_KEY"

# Define evaluation agent
greeting_agent = Agent(
    name="Greeting agent",
    instructions="You only evaluate greeting script in Thai. Give a score out of 10. Expected output format: JSON format with key equals to 'greeting' and value as string type",
)

# Define evaluation agent
script_adherence_agent = Agent(
    name="Script adherence agent",
    instructions="You only evaluate script adherence in Thai. Give a score out of 10. Expected output format: JSON format with key equals to 'script_adherence' and value as string type",
)

# Define evaluation agent
politeness_agent = Agent(
    name="Polite agent",
    instructions="You only evaluate politeness in Thai. Give a score out of 10. Expected output format: JSON format with key equals to 'politeness' and value as string type",
)

# Define evaluation agent
resolution_agent = Agent(
    name="Resolution agent",
    instructions="You only evaluate resolution in Thai. Give a score out of 10. Expected output format: JSON format with key equals to 'resolution' and value as string type",
)

# Define evaluation agent
evaluation_agent = Agent(
    name="Evaluation agent",
    instructions="Handoff to the appropriate agent based on the service of the request.",
    handoffs=[greeting_agent, script_adherence_agent, politeness_agent, resolution_agent],
)

async def main():
    # result = await Runner.run(triage_agent, input="สวัสดีครับ")
    result = await Runner.run(evaluation_agent, input=transcript + "service: polite")    
    # Display the output
    print(result.final_output)

if __name__ == "__main__":
    await main()

In [ ]:
# Audio processing and preprocessing
audio = AudioSegment.from_file("มิจฉาชีพ.wav")
audio = audio.set_channels(1).set_frame_rate(16000)
audio = effects.normalize(audio)
audio.export("normalized.wav", format="wav")

# Audio processing and preprocessing
y, sr = librosa.load("normalized.wav", sr=16000)
reduced = nr.reduce_noise(y=y, sr=sr)
sf.write("preprocessed_มิจฉาชีพ.wav", reduced, sr)
audio = AudioSegment.from_wav("preprocessed_มิจฉาชีพ.wav")

# Audio Slitting into smaller chunks
chunk_length_ms = 60 * 1000  # 60 seconds
chunks = math.ceil(len(audio) / chunk_length_ms)

# model = whisper.load_model("base")
# model = whisper.load_model("small")
# model = whisper.load_model("medium")
model = whisper.load_model("large")
transcript = ""

overlap_ms = 2 * 1000

for i in range(chunks):
    if i == 0:
      chunk = audio[i*chunk_length_ms:(i+1)*chunk_length_ms]
    else:
      chunk = audio[i*chunk_length_ms - overlap_ms:(i+1)*chunk_length_ms]
    chunk_path = f"chunk_{i}.wav"
    chunk.export(chunk_path, format="wav")
    
    result = model.transcribe(chunk_path, language="th")
    transcript += f"{result['text']}"

# Display the output
print(transcript)

In [ ]:
prompt = f"""
The following is a messy, unstructured transcription of a phone call in Thai. It contains overlapping phrases, unclear speaker turns, and no punctuation.

Your task is to:
1. Structure the content as a readable conversation.
2. Add punctuation and line breaks to make it natural.
3. Keep the language in Thai and maintain the emotional tone.

Expected output format:
JSON format with key equals to 'transcription' and value as string type

Here is the transcription:
{transcript}
"""

# Perform local inference using Ollama
response = ollama.chat(
    model="llama3.2:latest",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that restructures messy Thai call transcriptions."},
        {"role": "user", "content": prompt}
    ]
)

# Display the output
print(response['message']['content'])
transcript = response['message']['content']

In [ ]:
prompt = f"""
You are a QA agent. Evaluate the following call transcript.
Transcript:
{transcript}

Check for:
1. Proper greeting in Thai
2. Script adherence
3. Politeness by saying "ครับ"/"ค่ะ" in the conversation
4. Resolution provided

Give a score out of 10 for each criteria.

Expected output format:
JSON format with
1. key equals to 'greeting' and value as string type
2. key equals to 'script_adherence' and value as string type
3. key equals to 'politeness' and value as string type
4. key equals to 'resolution' and value as string type
"""

# Perform local inference using Ollama
response = ollama.chat(
    model="llama3.2:latest",
    messages=[
        {"role": "system", "content": "You are a helpful assistant for contact center QA."},
        {"role": "user", "content": prompt}
    ]
)

# Display the output
print(response['message']['content'])

In [ ]:
# Set your key here or via environment variable
embedding = OpenAIEmbeddings(openai_api_key="YOUR_API_KEY")

documents = [
    "Our company handles with Thai customer",
    "All agents/staffs must greet every customer in Thai language with 'สวัสดี'",
    "Customers should be treated with empathy.",
    "At the end of every dialogue response by agents/staffs must be ended with 'ครับ' or 'ค่ะ'",
]

# Create FAISS index from documents
faiss_index = FAISS.from_texts(documents, embedding)

llm = ChatOpenAI(
    openai_api_key="YOUR_API_KEY",
    model_name="gpt-4",
    temperature=0
)

 qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=faiss_index.as_retriever()
 )

# Now augment the prompt using retrieved knowledge
# Prepare the query for evaluation
query = f"Evaluate this transcript (in Thai Language): {transcript}"
# Display the output
print(qa_chain.invoke(query))

In [ ]:
# Define evaluation prompts
agents = {
    "greeting": {
        "name": "Greeting agent",
        "instructions": "You only evaluate greeting script in Thai. Give a score out of 10. Expected output format: JSON format with key equals to 'greeting' and value as string type"
    },
    "script_adherence": {
        "name": "Script adherence agent",
        "instructions": "You only evaluate script adherence in Thai. Give a score out of 10. Expected output format: JSON format with key equals to 'script_adherence' and value as string type"
    },
    "politeness": {
        "name": "Polite agent",
        "instructions": "You only evaluate politeness in Thai. Give a score out of 10. Expected output format: JSON format with key equals to 'politeness' and value as string type"
    },
    "resolution": {
        "name": "Resolution agent",
        "instructions": "You only evaluate resolution in Thai. Give a score out of 10. Expected output format: JSON format with key equals to 'resolution' and value as string type"
    }
}

service = "politeness"
instruction = agents[service]["instructions"]

# Perform local inference using Ollama
response = ollama.chat(
    model="llama3.2:latest", 
    messages=[
        {"role": "system", "content": instruction},
        {"role": "user", "content": transcript}
    ]
)

# Display the output
print("🔍 Evaluation result:", response['message']['content'])